In [1]:
from quadmodel.inference.forward_model import forward_model
import os
import numpy as np
import matplotlib.pyplot as plt

### Forward modeling image flux ratios with quadmodel

In the following cell, we define the parameters we want to sample in the forward model, and set the lens data class

In [4]:
output_path = os.getcwd() + '/example_inference_output/'
job_index = 1
n_keep = 2
summary_statistic_tolerance = np.inf

from quadmodel.data.b1422 import B1422
from quadmodel.data.wgd2038 import WGD2038
from quadmodel.data.rxj0911 import RXJ0911
from quadmodel.data.he0435 import HE0435
from quadmodel.data.pg1115 import PG1115 
from quadmodel.data.wfi2033 import WFI2033
from quadmodel.data.psj1606 import PSJ1606
#lens_data = HE0435(macromodel_type='EPL_FREE_SHEAR')
lens_data = HE0435(macromodel_type='EPL_FREE_SHEAR_MULTIPOLE')
print(lens_data.m)
print(lens_data.zlens)
print(lens_data.zsource)
print(lens_data.macromodel_type)
realization_priors = {}
realization_priors['PRESET_MODEL'] = 'WDM'
realization_priors['log10_sigma_sub'] = ['UNIFORM', -1.5, -1.4]
realization_priors['LOS_normalization'] = ['FIXED', 1.]
realization_priors['log_mc'] = ['UNIFORM', 4.0, 4.1]
realization_priors['log_m_host'] = ['FIXED', 13.3]
realization_priors['log_mlow'] = ['FIXED', 6.0]

macromodel_priors = {}
macromodel_priors['m4_amplitude_prior'] = [np.random.normal, 0.0, 0.005]
#macromodel_priors['m3_amplitude_prior'] = [np.random.normal, 0.0, 0.001]
macromodel_priors['gamma_macro_prior'] = [np.random.uniform, 1.9, 2.2]

# FOR A CUSTOM SHEAR PRIOR:
# macromodel_priors['shear_strength_prior'] = [np.random.uniform, 0.05, 0.25]

# the present lenses also have built-in shear priors determined based on what values get accepted after running ABC;
# using a brooader prior, you will waste some time exploring parameter space that will get rejected
# shear_min, shear_max = lens_data.kwargs_macromodel['shear_amplitude_min'], lens_data.kwargs_macromodel['shear_amplitude_max']
# print(shear_min, shear_max)
# macromodel_priors['shear_strength_prior'] = [np.random.uniform, shear_min, shear_max]


[0.96  0.976 1.    0.65 ]
0.45
1.69
EPL_FREE_SHEAR_MULTIPOLE


### Run the simulation with seed 1 

In [5]:
job_index = 1
forward_model(output_path, job_index, lens_data, n_keep, realization_priors, 
              macromodel_priors, tolerance=summary_statistic_tolerance, 
                  verbose=True, test_mode=True, save_realizations=True, 
             readout_macromodel_samples=True, n_macro=4, 
             subtract_exact_mass_sheets=True, log_mlow_mass_sheet=6.0, random_seed=2,
             ray_tracing_optimization='default', rescale_grid_resolution=2.0,
             rescale_grid_size=1.0, index_lens_split=[0,1])

# new: 1.23, 0.89, 0.42
# old: 1.21, 0.83, 0.42

# new: 1.22, 0.67, 0.32
# old: 1.23, 0.67, 0.34

# new: 4.03, 0.80, 1.53
# old: 2.5767117058397084, 0.9149895514116598, 1.818035929833114

# new: 1.00, 1.25, 0.85
# old: 0.84, 1.08, 0.76

reading output to files: 
/Users/danielgilman/Code/quadmodel/notebooks/example_inference_output/job_1/parameters.txt
/Users/danielgilman/Code/quadmodel/notebooks/example_inference_output/job_1/fluxes.txt
starting with 0 samples accepted, 2 remain
existing magnifications:  None
samples remaining:  2
running simulation with a summary statistic tolerance of:  inf
importance weight for sample:  1.0
sample (from realization samples):  [-1.43788662  4.05291421]
realization contains 8713 halos.
['log10_sigma_sub', 'log_mc']
realization hyper-parameters:  [-1.43788662  4.05291421]
['source_size_pc']
source/lens parameters:  34.338954628600334
['a_m_4', 'gamma_macro', 'theta_E', 'center_x', 'center_y']
macromodel samples:  [-0.00373935  2.04827105  0.30053033 -1.94177022 -3.08997182]


keyword arguments for realization: 
preset model function:  <function WDM at 0x7f89b28e3790>
kwargs preset model:  {'sigma_sub': 0.036484918736406725, 'LOS_normalization': 1.0, 'log_mc': 4.052914209427704, 'log_m

KeyboardInterrupt: 

In [ ]:
job_index = 1
forward_model(output_path, job_index, lens_data, n_keep, realization_priors, 
              macromodel_priors, tolerance=summary_statistic_tolerance, 
                  verbose=True, test_mode=True, save_realizations=True, 
             readout_macromodel_samples=True, n_macro=4, 
             subtract_exact_mass_sheets=True, log_mlow_mass_sheet=7.0, random_seed=2,
             ray_tracing_optimization='STANDARD', rescale_grid_resolution=1.0,
             rescale_grid_size=1.0, index_lens_split=[0,1])

### Run with seed 2

In [ ]:
job_index = 2
forward_model(output_path, job_index, lens_data, n_keep, realization_priors, 
              macromodel_priors, tolerance=summary_statistic_tolerance, 
                  verbose=True, test_mode=True, save_realizations=True, 
             readout_kappagamma_statistics=False, readout_curvedarc_statistics=False, 
             readout_macromodel_samples=True, n_macro=4, diff_scale_list=[0.0001, 0.025, 0.1],
             subtract_exact_mass_sheets=True, log_mlow_mass_sheet=7.0, random_seed=1,
             ray_tracing_optimization='DECOUPLED_MULTI_PLANE', rescale_grid_resolution=2.0)

### Examine the output

The simulation output includes files named fluxes.txt, parameters.txt, and simulation_output_1, simulation_output_2, etc.

In [ ]:
f = open(output_path + 'job_'+str(job_index)+'/parameters.txt', 'r')
param_names = f.readlines()[0]
print('PARAMETER NAMES:')
print(param_names)
f.close()

accepeted_parameters = np.loadtxt(output_path + 'job_'+str(job_index)+'/parameters.txt', skiprows=1)
print('ACCEPTED PARAMETERS:')
print(accepeted_parameters)
# the first set of parameters are the ones specified in kwargs_realization (see cell #2), the rest are the source size, 
# macromodel parameters, and the last parameter is the summary statistic

accepeted_mags = np.loadtxt(output_path + 'job_'+str(job_index)+'/fluxes.txt')
print('\nACCEPTED MAGNIFICATIONS:')
print(accepeted_mags)

kappa_gamma_statistics = np.loadtxt(output_path + 'job_'+str(job_index)+'/kappa_gamma_statistics.txt', skiprows=1)
print('\nCONVERGENCE AND SHEAR: ')
print(kappa_gamma_statistics)
print(kappa_gamma_statistics.shape)

curved_arc_statistics = np.loadtxt(output_path + 'job_'+str(job_index)+'/curvedarc_statistics.txt', skiprows=1)
print('\nCURVED ARC STATISTICS: ')
print(curved_arc_statistics)
print(curved_arc_statistics.shape)

### Visualize accepeted realizations

The pickeled classes in simulation_output allow you to visualize the accepeted realizations

In [ ]:
import pickle
f = open(output_path + 'job_'+str(job_index)+'/simulation_output_2', 'rb')
simulation_output = pickle.load(f)
f.close()
x_image, y_image = simulation_output.data.x, simulation_output.data.y
lens_system = simulation_output.lens_system

npix = 150
rminmax = 1.25
_r = np.linspace(-rminmax, rminmax, npix)
xx, yy = np.meshgrid(_r, _r)
shape0 = xx.shape
lensmodel, kwargs_lens = lens_system.get_lensmodel()
lensmodel_macro, kwargs_macro = lens_system.get_macro_lensmodel(4)

kappa = lensmodel.kappa(xx.ravel(), yy.ravel(), kwargs_lens).reshape(shape0)
kappa_macro = lensmodel_macro.kappa(xx.ravel(), yy.ravel(), kwargs_lens).reshape(shape0)

# print(kappa)
# print(kappa_macro)

In [ ]:
import matplotlib.pyplot as plt

vminmax = 0.05
kappa_subs_nonlinear = kappa - kappa_macro
extent = [-rminmax, rminmax, -rminmax, rminmax]

fig = plt.figure(1)
fig.set_size_inches(8, 8)
plt.imshow(kappa_subs_nonlinear, vmin=-vminmax, vmax=vminmax, origin='lower', cmap='bwr', 
           extent=extent)
plt.scatter(x_image, y_image, color='k')
plt.show()

source_size_pc = accepeted_parameters[0, 4]
#lens_system.plot_images(x_image, y_image, source_size_pc, lensmodel, kwargs_lens)

### Check convergence, shear, and curved arc properties

In [ ]:
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
#from quadmodel.Solvers.light_fit_util import split_by_image_curvedarc, split_by_scale_curvedarc

def split_curved_arc_params(params, keep_scale=1):
    rs_scale1 = params[:,0:4]
    rs_scale2 = params[:,4:8]
    rs_scale3 = params[:,8:12]
    ts_scale1 = params[:,12:16]
    ts_scale2 = params[:,16:20]
    ts_scale3 = params[:,20:24]
    curv_scale1 = params[:,24:28]
    curv_scale2 = params[:,28:32]
    curv_scale3 = params[:,32:36]
    dir_scale1 = params[:,36:40]
    dir_scale2 = params[:,40:44]
    dir_scale3 = params[:,44:48]
    dtandtan_scale1 = params[:,48:52]
    dtandtan_scale2 = params[:,52:56]
    dtandtan_scale3 = params[:,56:60]
    if keep_scale==1:
        rs = rs_scale1
        ts = ts_scale1
        curv = curv_scale1
        direction = dir_scale1
        dtan_dtan = dtandtan_scale1
    elif keep_scale==2:
        rs = rs_scale2
        ts = ts_scale2
        curv = curv_scale2
        direction = dir_scale2
        dtan_dtan = dtandtan_scale2
    elif keep_scale==3:
        rs = rs_scale3
        ts = ts_scale3
        curv = curv_scale3
        direction = dir_scale3
        dtan_dtan = dtandtan_scale3
    image1 = np.array([rs[0], ts[0], curv[0], direction[0], dtan_dtan[0]])
    image2 = np.array([rs[1], ts[1], curv[1], direction[1], dtan_dtan[1]])
    image3 = np.array([rs[2], ts[2], curv[2], direction[2], dtan_dtan[2]])
    image4 = np.array([rs[3], ts[3], curv[3], direction[3], dtan_dtan[3]])
    return image1.T, image2.T, image3.T, image4.T
        
simulation_index = 1
f = open(output_path + 'job_'+str(job_index)+'/simulation_output_'+str(simulation_index), 'rb')
simulation_output = pickle.load(f)
f.close()
x_image, y_image = simulation_output.data.x, simulation_output.data.y
lens_system = simulation_output.lens_system
lensmodel, kwargs_lens = lens_system.get_lensmodel()

ext = LensModelExtensions(lensmodel)

kwargs_arc_image1 = ext.curved_arc_estimate(x_image[0], y_image[0], kwargs_lens, tan_diff=True)
kwargs_arc_image2 = ext.curved_arc_estimate(x_image[1], y_image[1], kwargs_lens, tan_diff=True)
kwargs_arc_image3 = ext.curved_arc_estimate(x_image[2], y_image[2], kwargs_lens, tan_diff=True)
kwargs_arc_image4 = ext.curved_arc_estimate(x_image[3], y_image[3], kwargs_lens, tan_diff=True)

image1, image2, image3, image4 = split_curved_arc_params(curved_arc_statistics)
print(image1[0,:])
print(kwargs_arc_image1)

In [ ]:
xi = 0.5
yi = 0.1
s = 0.3
_r = np.linspace(-s/2, s/2, 13)
xx, yy = np.meshgrid(_r, _r)
xx += xi
yy += yi
print(xx)

print(11%2)